Before running the next cell execute this command in the Anaconda prompt:<br>
"set GOOGLE_APPLICATION_CREDENTIALS=C:\Users\bdolicki\Documents\Git\multilingual-analysis\code\analysing_predictions\ReviewsTranslation-feb05629559c.json"<br>
Afterwards, execute "jupyter notebook" and go back to this notebook.

In [ ]:
# Imports the Google Cloud client library
from google.cloud import translate

# Instantiates a client
translate_client = translate.Client()

## Example translation

In [ ]:
target = 'en'
source = 'nl'
text = "Wat tekst die u wilt vertalen"
translation = translate_client.translate(text, target_language=target,source_language=source)

In [4]:
translation

{'translatedText': 'Some text you want to translate',
 'input': 'Wat tekst die u wilt vertalen'}

## Translate dataset and save

In [6]:
with open("test.txt",encoding="utf-8") as f:
    original_sents = f.read().strip().split("\n")
original_sents = [sent.split("\t")[1] for sent in original_sents]

In [9]:
import pandas as pd

In [13]:
translations = [ for sent in original_sents]

In [14]:
translations = translate_client.translate(original_sents[:5], target_language=target,source_language=source)

In [23]:
import time

In [24]:
time.sleep(100)

As of 19.08.2019 Google Translate API has a requests limits: 100,000 characters	per	100 seconds	project and user (see: https://cloud.google.com/translate/quotas). Thus I had to run it twice and then merge results.

To respect the limit, we will split the sentences into batches. It is faster to process a batch of multiple sentences than all of these sentences separately.

In [45]:
idxs = [0]
char_limit = 100000
num_chars = 0
total_chars = 0
for i, sent in enumerate(original_sents):
    if char_limit<num_chars+len(sent):
        idxs.append(i-1)
        num_chars = 0
    else:
        num_chars += len(sent)
    total_chars += len(sent)

In [46]:
len(idxs)

29

In [48]:
total_chars/char_limit

28.69273

**If my calculations are correct, the total number of chars i 2,8 million and 1M is 20 euros, so you would be charged for all of this about 60 euros.**

In [42]:
time.sleep(100)

In [43]:
translation = translate_client.translate(original_sents[idxs[i]:idxs[i+1]], target_language=target,source_language=source)

In [50]:
translations = []

for i in range(len(idxs)):
    print("Batch",i)
    translation = translate_client.translate(original_sents[idxs[i]:idxs[i+1]], target_language=target,source_language=source) 
    print("Rate exceeded - sleep for 100 seconds")
    time.sleep(110)
        
    translations += [text["translatedText"] for text in translation]
    

Batch 0
Rate exceeded - sleep for 100 seconds
Batch 1


Forbidden: 403 POST https://translation.googleapis.com/language/translate/v2: User Rate Limit Exceeded

In [ ]:
translations[:5]

In [10]:
test_with_translation = pd.DataFrame(data={"tokenized_sents":original_sents})